# Reporte de uso de MP para gestion de reposiscion

In [1]:
import MySQLdb
from datetime import datetime, timedelta
import pandas as pd

db = MySQLdb.connect(host="192.168.0.12",    # tu host, usualmente localhost
                     user="operario",         # tu usuario
                     passwd="",  # tu password
                     db="dbp8100")        # el nombre de la base de datos

# Debes crear un objeto Cursor. Te permitirá
# ejecutar todos los queries que necesitas
cur = db.cursor()

In [2]:
fecha_actual = datetime.now()
fecha_actual = fecha_actual.strftime("%Y-%m-%d")

In [3]:
fecha_ayer = datetime.now()
fecha_ayer = fecha_ayer.strftime('%Y-%m-%d')

In [12]:
fecha_actual = input("Ingrese la fecha actual (YYYY-MM-DD): ") or fecha_actual
fecha_ayer = input("Ingrese la fecha de ayer (YYYY-MM-DD): ") or fecha_ayer

In [ ]:
cur.execute("""
    SELECT 
        productox.Codigo AS Codigo, 
        productox.Nombre AS Producto, 
        SUM(dcaptura.Valor) AS Dosificado 
    FROM 
        dbp8100.dcaptura AS dcaptura
    JOIN 
        dbp8100.tareaseje AS tareaseje ON dcaptura.IDT = tareaseje.NroID
    JOIN 
        dbp8100.productox AS productox ON productox.NroID = dcaptura.IDP 
    WHERE 
        (tareaseje.Fecha = %s AND tareaseje.Hora >= '23:00:00') OR 
        (tareaseje.Fecha > %s AND tareaseje.Fecha <= %s AND tareaseje.Hora < '23:00:00')
    GROUP BY 
        productox.Nombre
""", (fecha_ayer, fecha_ayer, fecha_actual))

18

In [14]:
# Obtener los resultados
resultados = cur.fetchall()
# Obtener los nombres de las columnas
columnas = [desc[0] for desc in cur.description]

# Crear un DataFrame de pandas
df = pd.DataFrame(resultados, columns=columnas)
df['Dosificado'] = df['Dosificado'] * -1  # Convertir a negativo

In [15]:
df

,Codigo,Producto,Dosificado
0,A-04-05,AFRECHILLO DE TRIGO,-7673.00
1,B-07,AFRECHO ARROZ,-15046.00
2,C-09,BIONUCLEO BOVINO 1%,-413.50
3,C-12,CARBONATO DE CALCIO,-1804.00
4,A-06,CASCARA ARROZ,-6109.00
5,C-12,CONCHILLA,-130.40
6,DESPT,DESPARASITAN-T,-0.90
7,A-03,EXP SOJA,-10846.00
8,D-14,LIGNOBOND DD,-155.95
9,A-02,MAIZ,-31852.00
